In [27]:
import os, sys, psycopg2, pickle
import numpy as np
import warnings; warnings.filterwarnings("ignore")
from natsort import natsorted
from scipy import misc
from PIL import Image, ImageFilter

### Create database table for Regions Clicking feature

In [62]:
regions_dir = '../app/assets/images/2-Regions'
regions_subdirs = [d for d in os.listdir(regions_dir) if not d.startswith('.')]
regions_subdirs = [os.path.join(regions_dir, s, 'cyan') for s in regions_subdirs]
regions_subdirs = natsorted(regions_subdirs)[2:]
print(regions_subdirs[0])

#### Create blank dict (138, 621, 1406, x)

In [64]:
slices_dict = dict()
slice_shape = (621, 1406)
no_slices = 138
for i in range(0, no_slices):
    slices_dict[i] = np.ndarray(slice_shape, dtype=object)

#### Populate dict with vals for region matches

In [71]:
def populate_slices_dict(slices_dict, regions_subdirs):
    for i, subdir in enumerate(regions_subdirs):
        try:
            images = natsorted([os.path.join(subdir, image) \
                        for image in os.listdir(subdir)
                        if not image.startswith('.')])
        except:
            continue
        for sli_n, image in enumerate(images):
            im = misc.imread(image, 'L')
            for index, x in np.ndenumerate(im):
                if x:
                    matching_slis = slices_dict[sli_n][index]
                    if matching_slis == None:
                        matching_slis = list()
                    region_name = subdir.split('/')[5]
                    if region_name not in matching_slis:
                        matching_slis.append(region_name)
        print(subdir)

In [72]:
for i in range(len(regions_subdirs)):
    print(i)
    populate_slices_dict(slices_dict, [regions_subdirs[i]])
    if i % 5 == 0:
        try:
            slices_dict_pickle = pickle.load(open('slices_dict.p', 'rb'))
            slices_dict_pickle.update(slices_dict)
            pickle.dump(slices_dict_pickle, open('slices_dict.p', 'wb'))
        except:
            pickle.dump(slices_dict, open('slices_dict.p', 'wb'))


0
../app/assets/images/2-Regions/Forebrain - Diencephalon - Anterior group of the posterior tubercular vmat2 neurons/cyan
1
../app/assets/images/2-Regions/Forebrain - Diencephalon - Anterior preoptic dopaminergic cluster/cyan
2
../app/assets/images/2-Regions/Forebrain - Diencephalon - Anterior pretectum cluster of vmat2 Neurons/cyan
3
../app/assets/images/2-Regions/Forebrain - Diencephalon - Caudal Hypothalamus/cyan
4
../app/assets/images/2-Regions/Forebrain - Diencephalon - Dopaminergic Cluster 1 - ventral thalamic and periventricular posterior tubercular DA neurons/cyan
5
../app/assets/images/2-Regions/Forebrain - Diencephalon - Dopaminergic Cluster 2 - posterior tuberculum/cyan
6
../app/assets/images/2-Regions/Forebrain - Diencephalon - Dopaminergic Cluster 3 - hypothalamus/cyan
7
../app/assets/images/2-Regions/Forebrain - Diencephalon - Dopaminergic Cluster 4-5 - posterior tuberculum and hypothalamus/cyan
8
../app/assets/images/2-Regions/Forebrain - Diencephalon - Dopaminergic Clus

In [73]:
slices_dict_pickle = pickle.load(open('slices_dict.p', 'rb'))

In [74]:
print(slices_dict_pickle.keys())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137]


#### Init and fill PostgreSQL database with slices_dict vals

In [94]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [95]:
create_table_command = (
"""
    DROP TABLE IF EXISTS regionsClicking;
    CREATE TABLE regionsClicking (
        slice_id SERIAL PRIMARY KEY,
        slice_no INT NOT NULL,
        slices_vals text[][][]
    );
""")

In [96]:
try:
    cursor.execute(create_table_command)
    cursor.close()
    conn.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()

In [99]:
conn = psycopg2.connect(dbname='zbrain2db', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [ ]:
def create_insert_commands(nyt_articles_meta):
    keys = nyt_articles_meta.keys()
    for key in keys:
        article = nyt_articles_meta[key]
        insert_SQL = (
            "INSERT INTO articles "
            "(title, author, "
            "date, section, href, path) "
            "VALUES (%s, %s, %s, %s, %s, %s);")
        values = [article['title'], article['author'], article['date'], 
                      article['section'], article['href'], article['path']]
        yield (insert_SQL, values)

In [ ]:
insert_commands = list(create_insert_commands(nyt_articles_meta))

In [ ]:
def execute_inserts():
    conn = psycopg2.connect(dbname='nytlitedb', user='gqe', host='localhost', password='')
    cursor = conn.cursor()

    for insert_command in insert_commands:
        try:
            cursor.execute(insert_command[0], insert_command[1])
        except Exception as e:
            print(e)
        
    cursor.close()
    conn.commit()